# Collate demographic data for each LSOA

This notebook collates various data sources and saves as `collated_data.csv` in data folder. For each LSOA, it finds:
* **Hospital information** - yearly stroke admissions, travel time to and thrombolysis rate of nearest IVT, and travel time to nearest IVT (considering transfers or not)
* **Demographic information** - IMD, IoD Income/IDACI/IDAOIP, ethnicity, health, rural/urban, age

## Import libraries and set file paths

In [1]:
# Import required libraries
from dataclasses import dataclass
import geopandas as gpd
import numpy as np
import os
import pandas as pd
import zipfile

In [2]:
# Define file paths
@dataclass(frozen=True)
class Paths:
    '''Singleton object for storing paths to data and database.'''

    data = './data'
    travel_matrix_zip = 'lsoa_travel_time_matrix_calibrated.zip'
    travel_matrix_csv = 'lsoa_travel_time_matrix_calibrated.csv'
    admissions = 'admissions_2017-2019.csv'
    units = 'stroke_hospitals_2022.csv'
    inter_hospital = 'inter_hospital_time_calibrated.csv'

    demographic = './data/demographic/'
    welsh_lsoa = 'Welsh IMD 2019/Welsh IMD 2019/wales_lsoa_code_name.csv'
    welsh_imd = 'Welsh IMD 2019/Welsh IMD 2019/WIMD2019_Scores.csv'
    english_imd = 'English IMD 2019/English IMD 2019/IMD_2019.shp'
    deprivation = ('File_7a_-_All_IoD2019_Scores__Ranks__Deciles' +
                   '_and_Population_Denominators_3.csv')
    ethnicity = 'ethnicity.csv'
    health = 'general_health.csv'
    rural_urban = ('Rural_Urban_Classification_2011_of_Lower_Layer_Super' +
                   '_Output_Areas_in_England_and_Wales.csv')
    age_all = 'mid_2020_persons.csv'
    age_female = 'mid_2020_females.csv'
    age_male = 'mid_2020_males.csv'


paths = Paths()

## Hospital information

### Import admissions, units and travel times

#### Admissions

This is the average yearly admissions for each LSOA from 2017 to 2019.

In [3]:
data = pd.read_csv(os.path.join(paths.data, paths.admissions), index_col='area')
data.rename(columns={'Admissions': 'admissions'}, inplace=True)

data.head(2)

,admissions
area,
Welwyn Hatfield 010F,0.666667
Welwyn Hatfield 012A,4.000000


#### Units

This has a record of information about every hospital - including whether they provide thrombolysis (IVT) and thrombectomy (MT).

In [4]:
units = pd.read_csv(os.path.join(paths.data, paths.units))

units.head(2)

,Postcode,Hospital_name,Use_IVT,Use_MT,Use_MSU,Country,Strategic Clinical Network,Health Board / Trust,Stroke Team,SSNAP name,...,Thrombolysis,ivt_rate,Easting,Northing,long,lat,Neuroscience,30 England Thrombectomy Example,hospital_city,Notes
0,RM70AG,RM70AG,1,1,1,England,London SCN,Barking,Havering and Redbridge University Hospitals N...,Queens Hospital Romford HASU,...,117.0,11.9,551118,187780,0.179031,51.568647,1,0,Romford,NaN
1,E11BB,E11BB,1,1,1,England,London SCN,Barts Health NHS Trust,The Royal London Hospital,Royal London Hospital HASU,...,115.0,13.4,534829,181798,-0.058133,51.519018,1,1,Royal London,NaN


#### LSOA to hospital travel times

This is a matrix with travel times (in minutes) from every LSOA to every hospital.

In [5]:
# Unzip travel matrix folder and save the csv file to data path
travel_path = os.path.join(paths.data, paths.travel_matrix_zip)
with zipfile.ZipFile(travel_path, 'r') as zip_ref:
    zip_ref.extractall(paths.data)

# Import the csv file produced
travel_time = pd.read_csv(os.path.join(paths.data, paths.travel_matrix_csv),
                          index_col='LSOA')

# Remove any leading spaces in column titles
travel_time.columns = travel_time.columns.str.lstrip()

travel_time.head(2)

,B152TH,B714HJ,B95SS,BA13NG,BA214AT,BB23HH,BD96RJ,BH77DW,BL97TD,BN112DH,...,TN240LZ,TQ27AA,TR13LQ,TS198PE,TS43BW,WD180HB,WF14DG,WR51DD,WV100QP,YO318HE
LSOA,,,,,,,,,,,,,,,,,,,,,
Adur 001A,173.3,179.8,171.2,161.5,152.9,274.3,261.4,100.3,262.5,18.7,...,89.6,212.0,291.5,310.8,306.5,92.8,242.1,178.7,186.2,267.9
Adur 001B,173.3,179.8,172.3,161.5,152.9,274.3,261.4,99.2,263.6,18.7,...,90.6,212.0,290.4,311.9,306.5,92.8,242.1,178.7,186.2,267.9


#### Inter-hospital travel times

This is a matrix with travel times (in minutes) from each hospital to every other hospital.

In [6]:
inter_hospital_times = pd.read_csv(
    os.path.join(paths.data, paths.inter_hospital), index_col='from_postcode')

# Remove any leading spaces in column titles
inter_hospital_times.columns = inter_hospital_times.columns.str.lstrip()

inter_hospital_times.head(2)

,B152TH,B714HJ,B95SS,BA13NG,BA214AT,BB23HH,BD96RJ,BH77DW,BL97TD,BN112DH,...,TN240LZ,TQ27AA,TR13LQ,TS198PE,TS43BW,WD180HB,WF14DG,WR51DD,WV100QP,YO318HE
from_postcode,,,,,,,,,,,,,,,,,,,,,
B152TH,0.0,18.7,18.7,107.8,155.1,117.5,135.8,173.3,106.8,176.6,...,184.1,176.6,256.0,191.6,187.3,114.3,122.9,36.9,29.4,148.6
B714HJ,18.7,0.0,24.1,106.8,154.0,110.0,128.2,179.8,98.2,183.0,...,184.1,176.6,255.0,188.4,184.1,111.1,119.6,35.9,21.9,145.4


### Get postcodes of units providing IVT or MT

Get postcodes of unit in use for IVT

In [7]:
mask = units['Use_IVT'] == 1
ivt_units = list(units[mask]['Postcode'])
ivt_units[0:5]

['RM70AG', 'E11BB', 'SW66SX', 'SE59RW', 'BR68ND']

Get postcodes of units providing MT.

In [8]:
mask = units['Use_MT'] == 1
mt_units = list(units[mask]['Postcode'])
mt_units[0:5]

['RM70AG', 'E11BB', 'SW66SX', 'SE59RW', 'SW170QT']

### Travel time to nearest thrombolysis unit

Limit to units that use IVT

In [9]:
travel_time_ivt = travel_time[ivt_units]
travel_time_ivt.head(2)

,RM70AG,E11BB,SW66SX,SE59RW,BR68ND,HA13UJ,SW170QT,NW12BU,DE223NE,NN15BD,...,LL137TD,LL572PW,CF144XW,CF479DT,CF311RQ,SY231ER,SA148QF,SA312AF,SA612PZ,SA66NL
LSOA,,,,,,,,,,,,,,,,,,,,,
Adur 001A,85.3,87.4,76.7,74.5,67.0,87.4,69.2,86.4,191.6,131.5,...,248.5,321.6,192.7,217.4,206.6,297.9,238.9,255.0,287.2,230.3
Adur 001B,85.3,87.4,76.7,74.5,67.0,87.4,69.2,86.4,191.6,131.5,...,248.5,321.6,192.7,217.4,207.7,297.9,238.9,255.0,287.2,230.3


Get closest unit and time  for IVT (convert series to dataframes)

In [10]:
closest_ivt_unit = pd.DataFrame(travel_time_ivt.idxmin(axis=1), columns=['closest_ivt_unit'])
closest_ivt_unit_time = pd.DataFrame(travel_time_ivt.min(axis=1), columns=['closest_ivt_unit_time'])

Merge into admissions

In [11]:
data = data.merge(closest_ivt_unit, left_index=True, right_index=True, how='left')
data = data.merge(closest_ivt_unit_time, left_index=True, right_index=True,  how='left')
data.head(2)

,admissions,closest_ivt_unit,closest_ivt_unit_time
area,,,
Welwyn Hatfield 010F,0.666667,SG14AB,18.7
Welwyn Hatfield 012A,4.000000,SG14AB,19.8


### Travel time to nearest thrombectomy unit from home (ignoring transfers)

Limit to units that use MT.

In [12]:
travel_time_mt = travel_time[mt_units]
travel_time_mt.head(2)

,RM70AG,E11BB,SW66SX,SE59RW,SW170QT,NW12BU,NG72UH,CB20QQ,B152TH,CV22DX,...,NE14LP,HU32JZ,LS13EX,S102JF,BN25BE,BS105NB,PL68DH,OX39DU,SO166YD,CF144XW
LSOA,,,,,,,,,,,,,,,,,,,,,
Adur 001A,85.3,87.4,76.7,74.5,69.2,86.4,188.4,122.9,173.3,159.4,...,332.3,267.9,251.7,223.8,17.6,162.6,237.8,112.1,76.7,192.7
Adur 001B,85.3,87.4,76.7,74.5,69.2,86.4,188.4,122.9,173.3,159.4,...,333.4,267.9,251.7,224.9,18.7,162.6,237.8,112.1,76.7,192.7


Get closest unit and time for MT (converting series to dataframes).

In [13]:
closest_mt_unit = pd.DataFrame(travel_time_mt.idxmin(axis=1), columns=['closest_mt_unit'])
closest_mt_unit_time = pd.DataFrame(travel_time_mt.min(axis=1), columns=['closest_mt_unit_time'])

Merge into admissions.

In [14]:
data = data.merge(closest_mt_unit, left_index=True, right_index=True, how='left')
data = data.merge(closest_mt_unit_time, left_index=True, right_index=True, how='left')
data.head(2)

,admissions,closest_ivt_unit,closest_ivt_unit_time,closest_mt_unit,closest_mt_unit_time
area,,,,,
Welwyn Hatfield 010F,0.666667,SG14AB,18.7,NW12BU,36.9
Welwyn Hatfield 012A,4.000000,SG14AB,19.8,NW12BU,36.9


### Travel time from closest IVT unit to closest MT unit.

<mark>To correct: needs to be transfer from closest IVT unit, to the MT unit that is closest to that IVT unit</mark>

In [15]:
# Loop through each LSOA, finding the closest IVT unit and closest MT unit
# Find the transfer time from the IVT unit to the MT unit
transfer_times = []
for index, value in data.iterrows():
    ivt_unit = value['closest_ivt_unit']
    mt_unit = value['closest_mt_unit']
    transfer_time = inter_hospital_times.loc[ivt_unit][mt_unit]
    transfer_times.append(transfer_time)

Merge into admissions

In [16]:
data['mt_transfer_time'] = transfer_times
data.head(2)

,admissions,closest_ivt_unit,closest_ivt_unit_time,closest_mt_unit,closest_mt_unit_time,mt_transfer_time
area,,,,,,
Welwyn Hatfield 010F,0.666667,SG14AB,18.7,NW12BU,36.9,46.6
Welwyn Hatfield 012A,4.000000,SG14AB,19.8,NW12BU,36.9,46.6


### Thrombolysis rate of closest IVT unit

In [17]:
# Get index from data (which is originally index from admissions file)
lsoa_index = data.index
lsoa_index

Index(['Welwyn Hatfield 010F', 'Welwyn Hatfield 012A', 'Welwyn Hatfield 002F',
       'Welwyn Hatfield 002E', 'Welwyn Hatfield 010A', 'Welwyn Hatfield 010B',
       'Welwyn Hatfield 012B', 'Welwyn Hatfield 011D', 'Welwyn Hatfield 011A',
       'Welwyn Hatfield 012C',
       ...
       'Cardiff 005F', 'Cardiff 049D', 'Cardiff 049E', 'Cardiff 049F',
       'Cardiff 005G', 'Cardiff 006F', 'Swansea 025F', 'Swansea 023E',
       'Swansea 025G', 'Swansea 025H'],
      dtype='object', name='area', length=34752)

In [18]:
# Get postcode and IVT rate of units that offer IVT
mask = units['Use_IVT'] == 1
ivt_rate = units[mask][['Postcode', 'ivt_rate']]

# Merge into admissions (adding rate for closest IVT unit to each LSOA)
data = data.merge(
    ivt_rate, left_on='closest_ivt_unit', right_on='Postcode', how='left')

# Drop the postcode column (duplicates closest_IVT_unit)
data.drop('Postcode', axis=1, inplace=True)

# Replace index (as was lost when merged)
data.set_index(lsoa_index, inplace=True)

In [19]:
data.head(2)

,admissions,closest_ivt_unit,closest_ivt_unit_time,closest_mt_unit,closest_mt_unit_time,mt_transfer_time,ivt_rate
area,,,,,,,
Welwyn Hatfield 010F,0.666667,SG14AB,18.7,NW12BU,36.9,46.6,6.8
Welwyn Hatfield 012A,4.000000,SG14AB,19.8,NW12BU,36.9,46.6,6.8


## Demographic information

### Index of multiple deprivation (IMD)

These are imported seperately:
* English - IMD scores extracted from shapefile
* Welsh - csv file with IMD scores has uncommon LSOA names, so the common LSOA names are added (as from 'wales_lsoa_code_name.csv' which was created from https://datamap.gov.wales/layers/appdata-ons:lsoa_wales_2011)

They are then joined and added to the main dataframe.

In [20]:
# Import common Welsh LSOA
wales_lsoa = pd.read_csv(os.path.join(paths.demographic, paths.welsh_lsoa),
                         index_col='LSOA11Code')

# Import welsh IMD scores
imd_wales = pd.read_csv(os.path.join(paths.demographic, paths.welsh_imd),
                        index_col='LSOA_Code')

# Add common LSOA name to the dataframe with Welsh IMD scores
imd_wales = pd.merge(imd_wales, wales_lsoa, left_index=True, right_index=True,  how='left')

# Use common LSOA name as index
imd_wales.set_index('lsoa11name', inplace=True)

# Set index as LSOA, then just keep the index and IMD column (renamed)
imd_wales = imd_wales[['WIMD2019_Score']]
imd_wales = imd_wales.rename(columns={'WIMD2019_Score': 'IMD2019Score'})

imd_wales.head()

,IMD2019Score
lsoa11name,
Isle of Anglesey 007A,19.2
Isle of Anglesey 007B,12.2
Isle of Anglesey 001A,26.8
Isle of Anglesey 001B,15.5
Isle of Anglesey 005A,16.0


In [21]:
# Import dataframe with English LSOA and IMD
imd_eng = gpd.read_file(os.path.join(paths.demographic, paths.english_imd),
                        crs='EPSG:27700')

# Set index as LSOA, then just keep the index and IMD column (renamed)
imd_eng.set_index('lsoa11nm', inplace=True)
imd_eng = imd_eng[['IMDScore']]
imd_eng = imd_eng.rename(columns={'IMDScore': 'IMD2019Score'})

imd_eng.head()

,IMD2019Score
lsoa11nm,
City of London 001A,6.208
City of London 001B,5.143
City of London 001C,19.402
City of London 001E,28.652
Barking and Dagenham 016A,19.837


In [22]:
# Create list containing the two dataframes
gdf_list = [imd_eng, imd_wales]

# Combine two dataframes and convert into GeoDataFrame
imd_eng_wales = gpd.GeoDataFrame(pd.concat(gdf_list))

# Merge on index (which is the LSOA)
data = data.merge(imd_eng_wales, left_index=True, right_index=True,  how='left')

data.head(2)

,admissions,closest_ivt_unit,closest_ivt_unit_time,closest_mt_unit,closest_mt_unit_time,mt_transfer_time,ivt_rate,IMD2019Score
area,,,,,,,,
Welwyn Hatfield 010F,0.666667,SG14AB,18.7,NW12BU,36.9,46.6,6.8,15.616
Welwyn Hatfield 012A,4.000000,SG14AB,19.8,NW12BU,36.9,46.6,6.8,33.313


### Deprivation

The [Indices of Deprivation (IoD)](https://assets.publishing.service.gov.uk/government/uploads/system/uploads/attachment_data/file/835115/IoD2019_Statistical_Release.pdf) is calculated from thirty indicators grouped into seven domains. One such domain is Income.

* **Income domain** - proportion of population experiencing deprivation relating to low levels of income
* **Income Deprivation Affecting Children Index (IDACI)** - proportion of children aged 0-15 living in income deprived households - subset of Income Deprivation domain
* **Income Deprivation Affecting Older People Index (IDAOPI)** - proportion of adults aged 60 or older who experience income deprivation - subset of Income Deprivation domain

In [23]:
# Import deprivation data
deprivation = pd.read_csv(os.path.join(paths.demographic, paths.deprivation),
                          index_col='LSOA name (2011)')

# Drop unwanted columns
cols_to_drop = ['LSOA Code (2011)','Local Authority District code (2019)']
deprivation.drop(cols_to_drop, axis=1, inplace=True)

deprivation.head(2)

,Local Authority District name (2019),Income Domain Score,Income Domain Rank (where 1 is most deprived),IDACI Score,IDACI Rank (where 1 is most deprived),IDAOPI Score,IDAOPI Rank (where 1 is most deprived)
LSOA name (2011),,,,,,,
City of London 001A,City of London,0.007,34740,0.006,34715,0.012,34729
City of London 001B,City of London,0.034,31762,0.037,31529,0.030,33839


In [24]:
# Merge into data
data = data.merge(deprivation, left_index=True, right_index=True,  how='left')

# Reset index back to LSOA (as lost when merged)
data.set_index(lsoa_index, inplace=True)

data.head(2)

,admissions,closest_ivt_unit,closest_ivt_unit_time,closest_mt_unit,closest_mt_unit_time,mt_transfer_time,ivt_rate,IMD2019Score,Local Authority District name (2019),Income Domain Score,Income Domain Rank (where 1 is most deprived),IDACI Score,IDACI Rank (where 1 is most deprived),IDAOPI Score,IDAOPI Rank (where 1 is most deprived)
area,,,,,,,,,,,,,,,
Welwyn Hatfield 010F,0.666667,SG14AB,18.7,NW12BU,36.9,46.6,6.8,15.616,Welwyn Hatfield,0.104,17062,0.125,17700,0.186,11760
Welwyn Hatfield 012A,4.000000,SG14AB,19.8,NW12BU,36.9,46.6,6.8,33.313,Welwyn Hatfield,0.176,9058,0.270,6329,0.269,6337


### Ethnicity

This is the number of people in each ethnic group in each LSOA.

In [25]:
# Import ethnicity data
ethnicity = pd.read_csv(os.path.join(paths.demographic, paths.ethnicity),
                        index_col='geography')

# Only get columns with ethnicity for all people (ie. no age/gender breakdown)
cols_to_include = [
    x for x in list(ethnicity) if 'Sex: All persons; Age: All categories' in x]
extract = ethnicity[cols_to_include]

# Truncate field name - removing "'Sex: All persons; Age: All categories: Age; "
# As this phrase is consistent for all, can just remove the first 44 characters
rename_dict = dict()
for field in list(extract):
    rename_dict[field] = field[44:]

# Rename the columns
extract = extract.rename(rename_dict, axis='columns')
    
# Merge in with data
data = data.merge(extract, left_index=True, right_index=True,  how='left')

# Reset index to LSOA (as lost on merge)
data.set_index(lsoa_index, inplace=True)

data.head(2)

,admissions,closest_ivt_unit,closest_ivt_unit_time,closest_mt_unit,closest_mt_unit_time,mt_transfer_time,ivt_rate,IMD2019Score,Local Authority District name (2019),Income Domain Score,...,Ethnic Group: Asian/Asian British: Bangladeshi; measures: Value,Ethnic Group: Asian/Asian British: Chinese; measures: Value,Ethnic Group: Asian/Asian British: Other Asian; measures: Value,Ethnic Group: Black/African/Caribbean/Black British: Total; measures: Value,Ethnic Group: Black/African/Caribbean/Black British: African; measures: Value,Ethnic Group: Black/African/Caribbean/Black British: Caribbean; measures: Value,Ethnic Group: Black/African/Caribbean/Black British: Other Black; measures: Value,Ethnic Group: Other ethnic group: Total; measures: Value,Ethnic Group: Other ethnic group: Arab; measures: Value,Ethnic Group: Other ethnic group: Any other ethnic group; measures: Value
area,,,,,,,,,,,,,,,,,,,,,
Welwyn Hatfield 010F,0.666667,SG14AB,18.7,NW12BU,36.9,46.6,6.8,15.616,Welwyn Hatfield,0.104,...,14,23,35,105,73,30,2,14,6,8
Welwyn Hatfield 012A,4.000000,SG14AB,19.8,NW12BU,36.9,46.6,6.8,33.313,Welwyn Hatfield,0.176,...,28,37,23,151,127,11,13,25,11,14


### Health

In [26]:
# Import health data
health = pd.read_csv(os.path.join(paths.demographic, paths.health),
                     index_col='geography')

# Only get columns with health for all people (ie. no age/gender breakdown)
cols_to_include = [
    x for x in list(health) if 'Sex: All persons; Age: All categories' in x]
extract = health[cols_to_include]

# Truncate field name - removing "'Sex: All persons; Age: All categories: Age; "
# As this phrase is consistent for all, can just remove the first 44 characters
rename_dict = dict()
for field in list(extract):
    rename_dict[field] = field[44:]
extract = extract.rename(rename_dict, axis='columns')
    
# Merge in with data
data = data.merge(extract, left_index=True, right_index=True,  how='left')

# Reset index to LSOA (as lost on merge)
data.set_index(lsoa_index, inplace=True)

data.head(2)

,admissions,closest_ivt_unit,closest_ivt_unit_time,closest_mt_unit,closest_mt_unit_time,mt_transfer_time,ivt_rate,IMD2019Score,Local Authority District name (2019),Income Domain Score,...,General Health: Fair health; Ethnic Group: Other ethnic group; measures: Value,General Health: Bad or very bad health; Ethnic Group: All categories: Ethnic group; measures: Value,General Health: Bad or very bad health; Ethnic Group: White: Total; measures: Value,General Health: Bad or very bad health; Ethnic Group: White: English/Welsh/Scottish/Northern Irish/British; measures: Value,General Health: Bad or very bad health; Ethnic Group: White: Irish; measures: Value,General Health: Bad or very bad health; Ethnic Group: White: Other White; measures: Value,General Health: Bad or very bad health; Ethnic Group: Mixed/multiple ethnic group; measures: Value,General Health: Bad or very bad health; Ethnic Group: Asian/Asian British; measures: Value,General Health: Bad or very bad health; Ethnic Group: Black/African/Caribbean/Black British; measures: Value,General Health: Bad or very bad health; Ethnic Group: Other ethnic group; measures: Value
area,,,,,,,,,,,,,,,,,,,,,
Welwyn Hatfield 010F,0.666667,SG14AB,18.7,NW12BU,36.9,46.6,6.8,15.616,Welwyn Hatfield,0.104,...,1,64,55,51,1,3,3,2,4,0
Welwyn Hatfield 012A,4.000000,SG14AB,19.8,NW12BU,36.9,46.6,6.8,33.313,Welwyn Hatfield,0.176,...,1,103,98,87,5,6,1,1,2,1


### Rural v.s. urban

In [27]:
# Import rural/urban data
rural_urban = pd.read_csv(os.path.join(paths.demographic, paths.rural_urban),
                          index_col='LSOA11NM')

# Drop unwanted columns
cols_to_drop = ['LSOA11CD','RUC11CD', 'FID']
rural_urban.drop(cols_to_drop, axis=1, inplace=True)

# Merge on LSOA
data = data.merge(rural_urban, left_index=True, right_index=True,  how='left')
data.set_index(lsoa_index, inplace=True)

data.head(2)

,admissions,closest_ivt_unit,closest_ivt_unit_time,closest_mt_unit,closest_mt_unit_time,mt_transfer_time,ivt_rate,IMD2019Score,Local Authority District name (2019),Income Domain Score,...,General Health: Bad or very bad health; Ethnic Group: All categories: Ethnic group; measures: Value,General Health: Bad or very bad health; Ethnic Group: White: Total; measures: Value,General Health: Bad or very bad health; Ethnic Group: White: English/Welsh/Scottish/Northern Irish/British; measures: Value,General Health: Bad or very bad health; Ethnic Group: White: Irish; measures: Value,General Health: Bad or very bad health; Ethnic Group: White: Other White; measures: Value,General Health: Bad or very bad health; Ethnic Group: Mixed/multiple ethnic group; measures: Value,General Health: Bad or very bad health; Ethnic Group: Asian/Asian British; measures: Value,General Health: Bad or very bad health; Ethnic Group: Black/African/Caribbean/Black British; measures: Value,General Health: Bad or very bad health; Ethnic Group: Other ethnic group; measures: Value,RUC11
area,,,,,,,,,,,,,,,,,,,,,
Welwyn Hatfield 010F,0.666667,SG14AB,18.7,NW12BU,36.9,46.6,6.8,15.616,Welwyn Hatfield,0.104,...,64,55,51,1,3,3,2,4,0,Urban city and town
Welwyn Hatfield 012A,4.000000,SG14AB,19.8,NW12BU,36.9,46.6,6.8,33.313,Welwyn Hatfield,0.176,...,103,98,87,5,6,1,1,2,1,Urban city and town


### Age

In [28]:
def age_bands(file_path, data, group, all_col, lsoa_index=lsoa_index):
    '''
    Finds count of individuals in each age band in each LSOA, using data
    provided (which has all persons, all males or all females). Adds these
    columns to data.
    Input:
    - file_path: string - path to age data
    - data: dataframe - contains hospital and demographic information by LSOA
    - group: string - group of people who the age data is from
    - all_col: string - name of column with all ages
    - lsoa_index: index to reset to
    Output:
    - data: dataframe - as before, but with added age counts
    '''
    # Import age data, setting index as LSOA
    ages = pd.read_csv(file_path, index_col='LSOA Name')

    # Save total person count to data
    data[all_col] = ages['All Ages']

    # Drop 'All Ages' (NOTE: DO I NEED TO?)
    ages.drop('All Ages', axis=1, inplace=True)

    # Change 90+ to 90 (NOTE: DO I NEED TO?)
    ages = ages.rename({'90+': '90'}, axis='columns')

    # Create empty dataframe to store age bands
    age_bands = pd.DataFrame()

    # Loop through numbers in 5s (0, 5, 10, 15... 90, 95)
    for band in np.arange(0, 96,5):
        # Create empty list
        cols_to_get = []
        # Create lists with ages 0, 1, 2... 89, 90 as strings
        age_list = [str(x) for x in range(0,91)]
        for field in age_list:
            # Divide by 5 and convert to nearest int (e.g. int(14/5)=2)
            # Multiply by 5 to get the band - save if in current band of loop
            if int(int(field)/5) * 5 == band:
                cols_to_get.append(field)
        # Extract columns with ages in that band (e.g. for 0 - 0, 1, 2, 3, 4)
        extract = ages[cols_to_get]
        # Sum of people in that age band
        age_bands[f'age band {group}{band}'] = extract.sum(axis=1)

    # Merge age band sums with data on LSOA 
    data = data.merge(age_bands, left_index=True, right_index=True,  how='left')
    data.set_index(lsoa_index, inplace=True)

    return(data)

In [29]:
# For all people, add count in each age band
data = age_bands(os.path.join(paths.demographic, paths.age_all), data,
                 '', 'All persons')
data.head(2)

,admissions,closest_ivt_unit,closest_ivt_unit_time,closest_mt_unit,closest_mt_unit_time,mt_transfer_time,ivt_rate,IMD2019Score,Local Authority District name (2019),Income Domain Score,...,age band 50,age band 55,age band 60,age band 65,age band 70,age band 75,age band 80,age band 85,age band 90,age band 95
area,,,,,,,,,,,,,,,,,,,,,
Welwyn Hatfield 010F,0.666667,SG14AB,18.7,NW12BU,36.9,46.6,6.8,15.616,Welwyn Hatfield,0.104,...,115,102,74,72,55,44,22,10,9,0.0
Welwyn Hatfield 012A,4.000000,SG14AB,19.8,NW12BU,36.9,46.6,6.8,33.313,Welwyn Hatfield,0.176,...,122,127,87,47,45,39,29,20,23,0.0


In [30]:
# For all females, add count in each age band
data = age_bands(
    os.path.join(paths.demographic, paths.age_female), data,
    'females', 'All females')
data.head(2)

,admissions,closest_ivt_unit,closest_ivt_unit_time,closest_mt_unit,closest_mt_unit_time,mt_transfer_time,ivt_rate,IMD2019Score,Local Authority District name (2019),Income Domain Score,...,age band females50,age band females55,age band females60,age band females65,age band females70,age band females75,age band females80,age band females85,age band females90,age band females95
area,,,,,,,,,,,,,,,,,,,,,
Welwyn Hatfield 010F,0.666667,SG14AB,18.7,NW12BU,36.9,46.6,6.8,15.616,Welwyn Hatfield,0.104,...,49,43,46,39,27,18,8,5,6,0.0
Welwyn Hatfield 012A,4.000000,SG14AB,19.8,NW12BU,36.9,46.6,6.8,33.313,Welwyn Hatfield,0.176,...,62,47,34,23,27,18,17,15,19,0.0


In [31]:
# For all males, add count in each age band
data = age_bands(
    os.path.join(paths.demographic, paths.age_male), data,
    'males ', 'All males')
data.head(2)

,admissions,closest_ivt_unit,closest_ivt_unit_time,closest_mt_unit,closest_mt_unit_time,mt_transfer_time,ivt_rate,IMD2019Score,Local Authority District name (2019),Income Domain Score,...,age band males 50,age band males 55,age band males 60,age band males 65,age band males 70,age band males 75,age band males 80,age band males 85,age band males 90,age band males 95
area,,,,,,,,,,,,,,,,,,,,,
Welwyn Hatfield 010F,0.666667,SG14AB,18.7,NW12BU,36.9,46.6,6.8,15.616,Welwyn Hatfield,0.104,...,66,59,28,33,28,26,14,5,3,0.0
Welwyn Hatfield 012A,4.000000,SG14AB,19.8,NW12BU,36.9,46.6,6.8,33.313,Welwyn Hatfield,0.176,...,60,80,53,24,18,21,12,5,4,0.0


## Save output

In [32]:
data.to_csv(f'./data/collated_data.csv', index_label='LSOA')